In [1]:
! pip install semantic-kernel -U
! pip install qdrant_client -U

In [2]:
import os
import json

In [3]:
import semantic_kernel as sk
from dotenv import load_dotenv
import semantic_kernel.connectors.ai.open_ai as skaoai

In [4]:
kernel = sk.Kernel()
deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()

In [5]:
kernel.add_chat_service("azure_chat_competion_service", skaoai.AzureChatCompletion("Your Azure OpenAI Service gpt-4-32k Model Delployment Name",endpoint,api_key=api_key,api_version = "2023-12-01-preview"))


kernel.add_text_embedding_generation_service(
        "embeddings_services", skaoai.AzureTextEmbedding("Your Azure OpenAI Service text-embedding-ada-002 Model Delployment Name", endpoint,api_key=api_key,api_version = "2023-12-01-preview")
)

Kernel(plugins=KernelPluginCollection(plugins={}), prompt_template_engine=PromptTemplateEngine(), memory=NullMemory(), text_completion_services={'azure_chat_competion_service': <function Kernel.add_text_completion_service.<locals>.<lambda> at 0x1240b0ca0>}, chat_services={'azure_chat_competion_service': <function Kernel.add_chat_service.<locals>.<lambda> at 0x12407e8c0>}, text_embedding_generation_services={'embeddings_services': <function Kernel.add_text_embedding_generation_service.<locals>.<lambda> at 0x1240b2cb0>}, default_text_completion_service='azure_chat_competion_service', default_chat_service='azure_chat_competion_service', default_text_embedding_generation_service='embeddings_services', retry_mechanism=PassThroughWithoutRetry(), function_invoking_handlers={}, function_invoked_handlers={})

In [6]:
base_plugin = "./plugins"

In [7]:
files_plugin = kernel.import_semantic_plugin_from_directory(base_plugin , "FilePlugin")

In [8]:
nodes_files = os.listdir("./data/notes")
transcrips_files = os.listdir("./data/transcripts")

In [9]:
kblist = []

In [10]:
for f in nodes_files:
    file = open("./data/notes/"+f, "r") 
    content = file.read()
    notesFunc = files_plugin["Notes"]
    result = await notesFunc(content)
    print(result.result.replace("\n", " ")) 
    json_result = json.loads(result.result)
    kblist.append(json_result)
    file.close()

{"kb": "History of machine learning", "content": "The history of machine learning and artificial intelligence is marked by notable discoveries such as Bayes Theorem (1763, 1812), Least Square Theory (1805), Markov Chains (1913), and Perceptron (1957). The concept of a 'machine that can think' was introduced by Alan Turing in 1950. The term 'artificial intelligence' was coined during the Dartmouth Summer Research Project in 1956. The period from 1956 to 1974 is often referred to as 'The golden years' of AI. However, by mid-1970s, the complexity of creating 'intelligent machines' led to a period known as 'AI Winter'. The 1980s saw the rise of 'expert systems'. The period from 1993 to 2011 marked a new era for ML and AI with the increase in data and compute power. Today, machine learning and AI touch almost every part of our lives."}
{ "kb": "Techniques of Machine Learning", "content": "The process of machine learning involves several steps: deciding on the question, collecting and prepar

In [11]:
for f in transcrips_files:
    file = open("./data/transcripts/"+f, "r") 
    content = file.read()
    transcripsFunc = files_plugin["Transcrips"]
    result = await transcripsFunc(content)
    print(result.result)
    json_result = json.loads(result.result)
    for item in json_result:
        kblist.append(item)
    file.close()

[
{"kb": "Alan Turing and the Turing test", "content": "Alan Turing, foundation, machine thinking, Turing test, intelligent computer"},
{"kb": "The Dartmouth Summer Research Project on AI", "content": "Artificial intelligence, Dartmouth College, summer research, AI birth, research field"},
{"kb": "The golden years of AI", "content": "AI optimism, natural language processing, micro worlds, intelligent machines, government funding"},
{"kb": "The AI winter", "content": "Complexity, understated promise, limited compute power, ethics questions, funding dried"},
{"kb": "Resurgence and fall of AI for expert systems", "content": "Expert systems, AI optimism, practical applications, too specialized, personal computers"},
{"kb": "Growth in AI driven by more data and more powerful hardware", "content": "Compute and storage capabilities, internet rise, smartphones, large data sets, machine learning"},
{"kb": "Big Data", "content": "Compute power, data sets growth, machine learning, problem solving

In [12]:
kblist

[{'kb': 'History of machine learning',
  'content': "The history of machine learning and artificial intelligence is marked by notable discoveries such as Bayes Theorem (1763, 1812), Least Square Theory (1805), Markov Chains (1913), and Perceptron (1957). The concept of a 'machine that can think' was introduced by Alan Turing in 1950. The term 'artificial intelligence' was coined during the Dartmouth Summer Research Project in 1956. The period from 1956 to 1974 is often referred to as 'The golden years' of AI. However, by mid-1970s, the complexity of creating 'intelligent machines' led to a period known as 'AI Winter'. The 1980s saw the rise of 'expert systems'. The period from 1993 to 2011 marked a new era for ML and AI with the increase in data and compute power. Today, machine learning and AI touch almost every part of our lives."},
 {'kb': 'Techniques of Machine Learning',
  'content': "The process of machine learning involves several steps: deciding on the question, collecting and 

In [13]:
base_vectordb = 'aboutMLKBDemoDemo'

In [14]:

from semantic_kernel.connectors.memory.qdrant import QdrantMemoryStore

In [15]:
qdrant_store = QdrantMemoryStore(vector_size=1536, url="http://localhost",port=6333)
await qdrant_store.create_collection(base_vectordb)

In [16]:
kernel.register_memory_store(memory_store=qdrant_store)

In [17]:
import uuid

In [18]:
uuid.uuid4()

UUID('9e8d659c-32de-4306-aa6c-876c14112446')

In [19]:
for item in kblist:
    content = item["kb"] + ' - ' + item["content"]
    id =str(uuid.uuid4())
    await kernel.memory.save_information(base_vectordb, id=id, text=content)

In [20]:
ask = "can you tell me what is different ML and AI"

memories = await kernel.memory.search(
    base_vectordb, ask, limit=1, min_relevance_score=0.8
)

In [21]:
result = ''
for memory in memories:
    print(f"Top Result: {memory.text} with score {memory.relevance}")
    result = memory.text

Top Result: The difference between AI and ML - AI accomplishes human tasks, ML subset of AI, ML uses algorithms with score 0.8766147


In [22]:
answer_plugin = kernel.import_semantic_plugin_from_directory(base_plugin , "AnswerPlugin")

In [23]:
answerFunc = answer_plugin["Summary"]

In [24]:
summary_result = await answerFunc(result)

In [25]:
summary_result.result

'AI, or Artificial Intelligence, is a broad concept that involves machines performing tasks in a way that mimics human intelligence. Machine Learning (ML), on the other hand, is a subset of AI. It involves the use of algorithms to allow machines to learn from data and improve their performance over time. In simple terms, while AI is about making machines smart, ML is about making machines learn from experience.'